In [38]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import Chrome
from selenium import webdriver
import re
import unidecode
import time
import pandas as pd
import datetime
import uuid
from tqdm import tqdm
from datetime import date
from selenium.webdriver.chrome.options import Options

In [2]:
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183


[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


In [29]:
def obtain_links_periodos(driver):
    links_asistencias = []
    for i in driver.find_element_by_css_selector('table').find_elements_by_css_selector('a'):
        link = i.get_attribute('href')
        if any(re.findall('votaciones', link)):
            links_asistencias.append(link)
    return(links_asistencias)

def obtain_links_votaciones(driver):
    links_asistencias_sesion = []
    for i in driver.find_element_by_css_selector('table').find_elements_by_css_selector('a'):
        link = i.get_attribute('href')
        if any(re.findall('estadistico_votacion', link)):
            links_asistencias_sesion.append(link)
    return(links_asistencias_sesion)

def obtain_partyVotaciones(driver):
    links_asistencias_sesion = []
    partidos = []
    for i in driver.find_element_by_css_selector('table').find_elements_by_css_selector('a'):
        link = i.get_attribute('href')
        if any(re.findall('listados_votaciones', link)):
            links_asistencias_sesion.append(link)
            partidos.append(i.text)
    return(links_asistencias_sesion, partidos)

In [46]:
def month_to_num(x):
    if x == 'Enero':
        return(1)
    elif x == 'Febrero':
        return(2)
    elif x == 'Marzo':
        return(3)
    elif x == 'Abril':
        return(4)
    elif x == 'Mayo':
        return(5)
    elif x == 'Junio':
        return(6)
    elif x == 'Julio':
        return(7)
    elif x == 'Agosto':
        return(8)
    elif x == 'Septiembre':
        return(9)
    elif x == 'Octubre':
        return(10)
    elif x == 'Noviembre':
        return(11)
    elif x == 'Diciembre':
        return(12)

def obtain_votacion(driver, legislatura, partido):
    temp = driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(4) > td > table > tbody > tr:nth-child(2) > td > b').text.split('\n')[1].split(' ')[-1:][0]
    date_session = datetime.date(int(temp.split('-')[2]), month_to_num(temp.split('-')[1]), int(temp.split('-')[0]))
    titulo_votacion = driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(4) > td > table > tbody > tr:nth-child(2) > td > b').text.split('\n')[2]
    
    
    year_legis = []
    month_legis = []
    day_legis = []
    members = []
    party = []
    sentido_votacion = []
    legis = []
    titulo = []
    for i in driver.find_element_by_css_selector('table').find_elements_by_css_selector('tr')[10:]:
        try:
            if any(re.findall('[a-z]+', i.find_elements_by_css_selector('td')[0].text)):
                continue
            else:
                members.append(unidecode.unidecode(i.find_elements_by_css_selector('td')[1].text))
                sentido_votacion.append(i.find_elements_by_css_selector('td')[2].text)
                titulo.append(titulo_votacion)
                party.append(partido)
                year_legis.append(date_session.year)
                month_legis.append(date_session.month)
                day_legis.append(date_session.day)
                legis.append(legislatura)
        except:
            continue
            
    return(pd.DataFrame({'legislatura':legis, 
                 'year_session':year_legis, 
                 'month_session':month_legis, 
                 'day_session':day_legis, 
                 'member':members, 
                 'party':party, 
                 'voto':sentido_votacion,
                         'titulo_votacion':titulo
                 }))



In [45]:
links_legislaturas = [
    'http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_periodonplxiv.php',
    'http://sitllxiii.diputados.gob.mx/votaciones_por_periodonplxiii.php',
    'http://sitllxii.diputados.gob.mx/votaciones_por_periodonplxii.php', 
    'http://sitllxi.diputados.gob.mx/votaciones_por_periodonplxi.php', 
    'http://sitllx.diputados.gob.mx/votaciones_por_periodonp.php'
]

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install(), options = options)
for link, leg in tqdm(zip(links_legislaturas, [64, 63, 62, 61, 60]), total = 5):
    data_legislatura = []
    driver.get(link)
    links_periodos = obtain_links_periodos(driver)
    for link2 in tqdm(links_periodos):
        driver.get(link2)
        links_sesiones = obtain_links_votaciones(driver)
        for link3 in tqdm(links_sesiones):
            driver.get(link3)
            links_sesiones_partido, partidos = obtain_partyVotaciones(driver)
            for link4, p in tqdm(zip(links_sesiones_partido, partidos), total = len(links_sesiones_partido)):
                driver.get(link4)
                data_legislatura.append(obtain_votacion(driver, leg, p))
    df_attendance_leg = pd.concat(data_legislatura)
    name_file = 'roll_call' + str(leg) + '.csv'
    df_attendance_leg.to_csv('../../data/02-roll_call_votes/' + name_file, encoding = 'utf-8')

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:40<05:24, 40.54s/it]


 22%|██▏       | 2/9 [00:53<03:46, 32.35s/it]


 33%|███▎      | 3/9 [01:03<02:34, 25.69s/it]


 44%|████▍     | 4/9 [01:14<01:45, 21.09s/it]


 56%|█████▌    | 5/9 [01:20<01:05, 16.49s/it]


 67%|██████▋   | 6/9 [01:24<00:38, 12.84s/it]


 78%|███████▊  | 7/9 [01:26<00:19,  9.75s/it]


 89%|████████▉ | 8/9 [01:29<00:07,  7.48s/it]


100%|██████████| 9/9 [01:29<00:00,  9.99s/it]


  3%|▎         | 1/37 [01:30<54:20, 90.56s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:48<06:25, 48.21s/it]


 22%|██▏       | 2/9 [01:03<04:27, 38.23s/it]


 33%|███▎      | 3/9 [01:10<02:54, 29.10s/it]


 44%|████▍     | 4/9 [01:19<01:55, 23.04s/it]


 56%|█████▌    | 5/9 [01:25<01:11, 17.76s/it]


 67%|██████▋   | 6/9 [01:29<00:41, 13.72s/it]


 78%|███████▊  | 7/9 [01:32<00:20, 10.43s/it]


 89%|████████▉ | 8/9 [0

  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:39<05:17, 39.70s/it]


 22%|██▏       | 2/9 [00:56<03:50, 32.88s/it]


 33%|███▎      | 3/9 [01:06<02:35, 25.87s/it]


 44%|████▍     | 4/9 [01:12<01:40, 20.08s/it]


 56%|█████▌    | 5/9 [01:16<01:00, 15.20s/it]


 67%|██████▋   | 6/9 [01:19<00:34, 11.65s/it]


 78%|███████▊  | 7/9 [01:21<00:17,  8.71s/it]


 89%|████████▉ | 8/9 [01:23<00:06,  6.66s/it]


100%|██████████| 9/9 [01:24<00:00,  9.38s/it]


 38%|███▊      | 14/37 [21:20<34:20, 89.58s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:39<05:13, 39.15s/it]


 22%|██▏       | 2/9 [00:52<03:40, 31.53s/it]


 33%|███▎      | 3/9 [01:01<02:28, 24.77s/it]


 44%|████▍     | 4/9 [01:09<01:38, 19.62s/it]


 56%|█████▌    | 5/9 [01:13<00:59, 14.95s/it]


 67%|██████▋   | 6/9 [01:17<00:34, 11.60s/it]


 78%|███████▊  | 7/9 [01:19<00:17,  8.73s/it]


 89%|████████▉ | 8/9 [01:21<00:06,  6.68s/it]


100%|██████████| 9/9 [01:22<00:00,  9.15s/it]


 41%|█

 22%|██▏       | 2/9 [00:51<03:31, 30.19s/it]


 33%|███▎      | 3/9 [01:00<02:23, 23.90s/it]


 44%|████▍     | 4/9 [01:08<01:35, 19.12s/it]


 56%|█████▌    | 5/9 [01:13<00:59, 14.79s/it]


 67%|██████▋   | 6/9 [01:17<00:34, 11.52s/it]


 78%|███████▊  | 7/9 [01:19<00:17,  8.75s/it]


 89%|████████▉ | 8/9 [01:21<00:06,  6.67s/it]


100%|██████████| 9/9 [01:22<00:00,  9.13s/it]


 73%|███████▎  | 27/37 [40:27<14:46, 88.66s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:42<05:42, 42.85s/it]


 22%|██▏       | 2/9 [00:55<03:56, 33.72s/it]


 33%|███▎      | 3/9 [01:05<02:40, 26.73s/it]


 44%|████▍     | 4/9 [01:14<01:47, 21.40s/it]


 56%|█████▌    | 5/9 [01:20<01:06, 16.68s/it]


 67%|██████▋   | 6/9 [01:24<00:38, 12.84s/it]


 78%|███████▊  | 7/9 [01:26<00:19,  9.65s/it]


 89%|████████▉ | 8/9 [01:28<00:07,  7.40s/it]


100%|██████████| 9/9 [01:29<00:00,  9.97s/it]


 76%|███████▌  | 28/37 [41:57<13:23, 89.24s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%

  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [01:53<15:08, 113.61s/it]


 22%|██▏       | 2/9 [02:17<10:06, 86.62s/it] 


 33%|███▎      | 3/9 [02:33<06:32, 65.47s/it]


 44%|████▍     | 4/9 [02:57<04:25, 53.06s/it]


 56%|█████▌    | 5/9 [03:08<02:42, 40.50s/it]


 67%|██████▋   | 6/9 [03:21<01:36, 32.13s/it]


 78%|███████▊  | 7/9 [03:31<00:51, 25.64s/it]


 89%|████████▉ | 8/9 [03:40<00:20, 20.53s/it]


100%|██████████| 9/9 [03:44<00:00, 24.93s/it]


  1%|          | 1/114 [03:45<7:05:30, 225.93s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [02:09<17:15, 129.46s/it]


 22%|██▏       | 2/9 [03:08<12:37, 108.19s/it]


 33%|███▎      | 3/9 [03:33<08:19, 83.29s/it] 


 44%|████▍     | 4/9 [03:48<05:15, 63.03s/it]


 56%|█████▌    | 5/9 [04:05<03:16, 49.20s/it]


 67%|██████▋   | 6/9 [04:17<01:53, 37.85s/it]


 78%|███████▊  | 7/9 [04:21<00:55, 27.73s/it]


 89%|████████▉ | 8/9 [04:24<00:20, 20.43s/it]


100%|██████████| 9/9 [04:26<00:00, 29.61s/it]


 22%|██▏       | 2/9 [02:29<10:31, 90.18s/it] 


 33%|███▎      | 3/9 [02:52<07:00, 70.09s/it]


 44%|████▍     | 4/9 [03:15<04:39, 55.95s/it]


 56%|█████▌    | 5/9 [03:31<02:55, 43.92s/it]


 67%|██████▋   | 6/9 [03:40<01:39, 33.29s/it]


 78%|███████▊  | 7/9 [03:45<00:49, 24.82s/it]


 89%|████████▉ | 8/9 [03:49<00:18, 18.72s/it]


100%|██████████| 9/9 [03:51<00:00, 25.77s/it]


 12%|█▏        | 14/114 [41:18<5:15:43, 189.43s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [01:54<15:12, 114.02s/it]


 22%|██▏       | 2/9 [02:14<10:02, 86.03s/it] 


 33%|███▎      | 3/9 [02:27<06:24, 64.06s/it]


 44%|████▍     | 4/9 [02:48<04:15, 51.17s/it]


 56%|█████▌    | 5/9 [02:59<02:36, 39.03s/it]


 67%|██████▋   | 6/9 [03:08<01:29, 29.94s/it]


 78%|███████▊  | 7/9 [03:13<00:44, 22.44s/it]


 89%|████████▉ | 8/9 [03:17<00:17, 17.04s/it]


100%|██████████| 9/9 [03:19<00:00, 22.15s/it]


 13%|█▎        | 15/114 [44:39<5:18:03, 192.76s/it]


  0%|          | 0/9 [00:00<?, ?i